# Pre-Procesamiento del Corpus
Obtención de los datos

Se agrega la extensión .txt a los documentos.

In [1]:
import os

# Ruta donde están los archivos
ruta_carpeta = "./data"

# Iterar sobre todos los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    # Ruta completa del archivo original
    archivo_original = os.path.join(ruta_carpeta, archivo)

    # Verificar que sea un archivo (no una carpeta)
    if os.path.isfile(archivo_original):
        # Separar el nombre y la extensión
        nombre, extension = os.path.splitext(archivo)

        # Crear el nuevo nombre con la extensión .txt
        nuevo_nombre = nombre + ".txt"
        archivo_nuevo = os.path.join(ruta_carpeta, nuevo_nombre)

        # Renombrar el archivo
        os.rename(archivo_original, archivo_nuevo)
        print(f"Archivo renombrado: {archivo} -> {nuevo_nombre}")


Archivo renombrado: 1.txt -> 1.txt
Archivo renombrado: 10.txt -> 10.txt
Archivo renombrado: 100.txt -> 100.txt
Archivo renombrado: 1000.txt -> 1000.txt
Archivo renombrado: 10000.txt -> 10000.txt
Archivo renombrado: 10002.txt -> 10002.txt
Archivo renombrado: 10005.txt -> 10005.txt
Archivo renombrado: 10008.txt -> 10008.txt
Archivo renombrado: 10011.txt -> 10011.txt
Archivo renombrado: 10014.txt -> 10014.txt
Archivo renombrado: 10015.txt -> 10015.txt
Archivo renombrado: 10018.txt -> 10018.txt
Archivo renombrado: 10023.txt -> 10023.txt
Archivo renombrado: 10025.txt -> 10025.txt
Archivo renombrado: 10027.txt -> 10027.txt
Archivo renombrado: 1003.txt -> 1003.txt
Archivo renombrado: 10032.txt -> 10032.txt
Archivo renombrado: 10035.txt -> 10035.txt
Archivo renombrado: 10037.txt -> 10037.txt
Archivo renombrado: 10038.txt -> 10038.txt
Archivo renombrado: 10040.txt -> 10040.txt
Archivo renombrado: 10041.txt -> 10041.txt
Archivo renombrado: 10042.txt -> 10042.txt
Archivo renombrado: 10043.txt -> 

Se importan las librerias necesarias.

In [2]:
import csv
import html

In [3]:
# Ruta de la carpeta con los archivos .txt
ruta_carpeta = "./data"

# Ruta del archivo cats.txt
ruta_categorias = "./cats.txt"

# Ruta de salida para el archivo CSV
ruta_csv = "database.csv"

Cargar las categorías desde cats.txt en un diccionario

In [4]:
categorias = {}
with open(ruta_categorias, "r") as f:
    for linea in f:
        partes = linea.strip().split(maxsplit=1)  # Dividir en nombre y categoría
        if len(partes) == 2:  # Verificar que tiene nombre y categoría
            nombre_archivo = partes[0].split("/")[-1]  # Extraer solo el número del archivo
            categorias[nombre_archivo] = partes[1]  # Guardar la categoría completa

Creación CSV database con las siguientes columnas:
- id
- documento
- titulo
- texto
- cats

In [5]:
# Crear el CSV
with open(ruta_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "document", "title", "text", "cats"])  # Escribir encabezado

    # Procesar cada archivo .txt en la carpeta
    for archivo in os.listdir(ruta_carpeta):
        if archivo.endswith(".txt"):  # Procesar solo archivos .txt
            ruta_archivo = os.path.join(ruta_carpeta, archivo)

            # Leer el contenido del archivo
            try:
                with open(ruta_archivo, "r", encoding="utf-8") as f:
                    lineas = f.readlines()
            except UnicodeDecodeError:
                with open(ruta_archivo, "r", encoding="latin-1") as f:
                    lineas = f.readlines()

            if lineas:  # Verificar que el archivo tiene contenido
                # Extraer campos
                nombre_archivo = os.path.splitext(archivo)[0]  # Quitar extensión
                documento = "".join(lineas).strip()  # Todo el contenido del archivo
                documento = html.unescape(documento)  # Decodificar entidades HTML
                titulo = lineas[0].strip()  # Primera línea como título
                texto = " ".join(lineas[1:]).strip()  # Texto excluyendo la primera línea
                categoria = categorias.get(nombre_archivo, "Sin categoría")  # Obtener categoría

                # Escribir fila en el CSV
                writer.writerow([nombre_archivo, documento, titulo, texto, categoria])

print(f"Archivo CSV creado en: {ruta_csv}")


Archivo CSV creado en: database.csv


# Procesamiento

In [6]:
import pandas as pd
import string
from nltk.stem import PorterStemmer
import re

In [7]:
# Cargar el archivo CSV
ruta_csv = "database.csv"
corpus = pd.read_csv(ruta_csv)

Se agrega la columna preproc_doc en la cual se hará el preprocesamiento del contenido de cada documento de la database.

In [8]:
corpus['preproc_doc'] = corpus['document']

En la función minusculas se realiza la conversión a minúsculas.

In [9]:
def minusculas(texto):
    
    texto = texto.lower()  # Convertir a minúsculas
    
    return texto


corpus['preproc_doc'] = corpus['preproc_doc'].apply(minusculas) 

En la función limpiar_texto se realiza la eliminación de caracteres especiales.

In [10]:
def limpiar_texto(texto):

    texto = re.sub(r'&lt;[^ ]*', '', texto)  # Eliminar el patrón desde &lt; hasta el primer espacio
    texto = re.sub(r'[<>]', '', texto)  # Remover caracteres sobrantes como < y >
    texto = re.sub(r'[.,;!?-]', '', texto)  # Remover caracteres específicos como , . ; ! ?
    texto = re.sub(r'[^\w\s]', '', texto)  # Remover cualquier otro carácter especial
    texto = re.sub(r'\s+', ' ', texto).strip()  # Reemplazar múltiples espacios por uno solo
    
    return texto


corpus['preproc_doc'] = corpus['preproc_doc'].apply(limpiar_texto) 
corpus['title'] = corpus['title'].apply(limpiar_texto)  # Limpiar título

Tokenización

In [11]:
def tokenizar_texto(texto):
    """Tokeniza el texto dividiéndolo en palabras."""
    return texto.split()

corpus['preproc_doc'] = corpus['preproc_doc'].apply(tokenizar_texto)  

Eliminación de stopwords.

In [12]:
# Cargar el archivo de stopwords personalizado
ruta_stopwords = "stopwords"  # Ruta al archivo de stopwords
with open(ruta_stopwords, "r", encoding="utf-8") as f:
    custom_stopwords = set(f.read().splitlines())  # Leer stopwords y convertir a conjunto

def eliminar_stopwords(tokens):
    """Elimina las stopwords del texto tokenizado usando el archivo stopwords."""
    return [word for word in tokens if word not in custom_stopwords]

corpus['preproc_doc'] = corpus['preproc_doc'].apply(eliminar_stopwords)

Stemming

In [13]:
def aplicar_stemming(tokens):
    """Aplica stemming a los tokens."""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

corpus['preproc_doc'] = corpus['preproc_doc'].apply(aplicar_stemming) 

Se guarda el corpus preprocesado en el mismo database.csv

In [14]:
corpus.to_csv(ruta_csv, index=False)

print("Corpus preprocesado guardado en 'database.csv'")


Corpus preprocesado guardado en 'database.csv'


# Vectorización

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

print("Todas las librerias se importaron correctamente.")

Todas las librerias se importaron correctamente.


Cargar el archivo database.csv

In [16]:
ruta_procesado = "database.csv"
corpus = pd.read_csv(ruta_procesado)

Converttir la lista de tokens de nuevo a texto y como listas para Word2Vec

In [17]:
corpus['tokens_list'] = corpus['preproc_doc'].apply(eval)  # Convertir strings de listas a listas reales
corpus['tokens_text'] = corpus['tokens_list'].apply(lambda x: " ".join(x))  # Convertir lista a texto

Bag of Words (BoW)

In [18]:
# Bag of Words (BoW)
bow_vectorizer = CountVectorizer()
bow_matrix = bow_vectorizer.fit_transform(corpus['tokens_text'])

# Guardar la matriz en archivo CSV
pd.DataFrame(bow_matrix.toarray(), columns=bow_vectorizer.get_feature_names_out()).to_csv("bow_matrix.csv", index=False)


TF-IDF

In [19]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus['tokens_text'])

# Guardar la matriz en archivo CSV
pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()).to_csv("tfidf_matrix.csv", index=False)

Word2Vec

In [20]:
# Word2Vec
# Entrenar el modelo Word2Vec con las listas de tokens
word2vec_model = Word2Vec(sentences=corpus['tokens_list'], vector_size=100, window=5, min_count=1, workers=4)

# Crear una representación vectorial promedio para cada documento
def get_average_word2vec(tokens, model, vector_size):
    """Obtiene el promedio de vectores de Word2Vec para un documento."""
    vector = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    return vector / count if count > 0 else vector

# Generar matriz de Word2Vec para el corpus
word2vec_matrix = np.array([get_average_word2vec(tokens, word2vec_model, 100) for tokens in corpus['tokens_list']])

# Guardar la matriz en archivo CSV
pd.DataFrame(word2vec_matrix).to_csv("word2vec_matrix.csv", index=False)


# Indexación

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar la base de datos preprocesada
database = pd.read_csv("database.csv")

# Cargar las matrices previamente generadas
bow_matrix = pd.read_csv("bow_matrix.csv")
tfidf_matrix = pd.read_csv("tfidf_matrix.csv")
word2vec_matrix = pd.read_csv("word2vec_matrix.csv")

# Crear un índice invertido basado en la matriz TF-IDF
def build_inverted_index(tfidf_matrix, feature_names):
    """Construir un índice invertido a partir de la matriz TF-IDF."""
    inverted_index = {}
    for term_idx, term in enumerate(feature_names):
        # Obtener los documentos que contienen el término
        docs_with_term = tfidf_matrix.iloc[:, term_idx].to_numpy().nonzero()[0]
        inverted_index[term] = docs_with_term.tolist()
    return inverted_index

# Cargar el texto procesado para entrenar el vectorizador TF-IDF
corpus = database["preproc_doc"]  # Usar la columna completa de texto preprocesado
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_transformed = tfidf_vectorizer.fit_transform(corpus)

# Convertir la matriz TF-IDF en DataFrame para su manipulación
tfidf_matrix_df = pd.DataFrame(
    tfidf_matrix_transformed.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

# Construir el índice invertido
feature_names = tfidf_vectorizer.get_feature_names_out()
inverted_index = build_inverted_index(tfidf_matrix_df, feature_names)

# Guardar el índice invertido
with open("inverted_index.json", "w") as f:
    json.dump(inverted_index, f)

print("Índice invertido creado y guardado como 'inverted_index.json'.")


Índice invertido creado y guardado como 'inverted_index.json'.


In [ ]:
import json

# Cargar el índice invertido
with open("inverted_index.json", "r") as f:
    inverted_index = json.load(f)

def buscar_termino(termino, indice):
    """
    Busca un término en el índice invertido y devuelve los documentos donde aparece.
    """
    if termino in indice:
        return indice[termino]
    else:
        return []

def buscar_consulta(consulta, indice):
    """
    Busca una consulta compuesta por varios términos en el índice invertido.
    Devuelve una lista de documentos donde aparecen todos los términos.
    """
    terminos = consulta.split()  # Dividir la consulta en términos
    resultados = [set(buscar_termino(termino, indice)) for termino in terminos]

    # Intersección de resultados (documentos que contienen todos los términos)
    if resultados:
        documentos_relevantes = set.intersection(*resultados)
        return list(documentos_relevantes)
    else:
        return []

# Ejemplo de búsqueda
consulta = "econo"
resultados = buscar_consulta(consulta, inverted_index)

print(f"Documentos relevantes para la consulta '{consulta}': {resultados}")


Documentos relevantes para la consulta 'automotive': []


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Cargar el corpus procesado
ruta_corpus = "database_preprocessed.csv"
corpus = pd.read_csv(ruta_corpus)

# Crear el vectorizador TF-IDF y ajustar al texto
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus["tokens_text"])  # Matriz TF-IDF

# Función de búsqueda con similitud coseno
def buscar_similitud_coseno(consulta, tfidf_vectorizer, tfidf_matrix, corpus, top_n=5):
    """
    Realiza una búsqueda basada en similitud coseno usando TF-IDF.
    
    Parámetros:
        consulta: Consulta de búsqueda (str).
        tfidf_vectorizer: Vectorizador TF-IDF ajustado al corpus.
        tfidf_matrix: Matriz TF-IDF del corpus.
        corpus: DataFrame con el corpus original.
        top_n: Número de documentos relevantes a devolver.
    
    Retorna:
        DataFrame con los documentos más relevantes y sus puntuaciones de similitud.
    """
    # Vectorizar la consulta
    consulta_vector = tfidf_vectorizer.transform([consulta])

    # Calcular similitud coseno entre la consulta y todos los documentos
    similitudes = cosine_similarity(consulta_vector, tfidf_matrix).flatten()

    # Obtener los índices de los documentos más similares
    top_indices = similitudes.argsort()[-top_n:][::-1]  # Top N en orden descendente

    # Crear un DataFrame con los resultados relevantes
    resultados = corpus.iloc[top_indices].copy()
    resultados["similitud"] = similitudes[top_indices]

    return resultados

# Ejemplo de búsqueda
consulta = "asian cocoa"
resultados = buscar_similitud_coseno(consulta, tfidf_vectorizer, tfidf_matrix, corpus, top_n=5)

# Mostrar los resultados
print("Resultados de búsqueda:")
print(resultados[["nombre del archivo", "similitud", "texto"]])

# Guardar resultados en un archivo CSV
resultados.to_csv("resultados_busqueda.csv", index=False)
print("Resultados guardados en 'resultados_busqueda.csv'.")


NameError: name 'limpiar_texto' is not defined

2.6. Evaluación del Sistema

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Resultados esperados (relevantes) para cada consulta (Ground Truth)
consultas = {
    "bahia cocoa": [0, 318, 3773, 4862, 7727],  # Índices de documentos relevantes para esta consulta
    "asian cocoa": [4862, 318, 7727, 319, 366],  # Otra consulta
}

# Función para calcular métricas de evaluación
def evaluar_sistema(consultas, tfidf_vectorizer, tfidf_matrix, corpus, top_n=5):
    resultados = []
    for consulta, relevantes_esperados in consultas.items():
        # Realizar búsqueda con similitud coseno
        consulta_vector = tfidf_vectorizer.transform([consulta])
        similitudes = cosine_similarity(consulta_vector, tfidf_matrix).flatten()
        indices_top = similitudes.argsort()[-top_n:][::-1]  # Índices de documentos más relevantes

        # Calcular métricas
        y_true = [1 if idx in relevantes_esperados else 0 for idx in range(len(corpus))]
        y_pred = [1 if idx in indices_top else 0 for idx in range(len(corpus))]

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        # Guardar los resultados de la consulta
        resultados.append({
            "consulta": consulta,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "relevantes_encontrados": indices_top.tolist(),
        })

    return resultados

# Evaluar el sistema
resultados_evaluacion = evaluar_sistema(consultas, tfidf_vectorizer, tfidf_matrix, corpus)

# Mostrar los resultados
for resultado in resultados_evaluacion:
    print(f"Consulta: {resultado['consulta']}")
    print(f"  Precisión: {resultado['precision']:.2f}")
    print(f"  Recuperación: {resultado['recall']:.2f}")
    print(f"  F1-Score: {resultado['f1_score']:.2f}")
    print(f"  Documentos encontrados: {resultado['relevantes_encontrados']}")
    print()

# Guardar los resultados en un archivo CSV
import pandas as pd
df_resultados = pd.DataFrame(resultados_evaluacion)
df_resultados.to_csv("evaluacion_sistema.csv", index=False)
print("Resultados de evaluación guardados en 'evaluacion_sistema.csv'.")


Consulta: bahia cocoa
  Precisión: 1.00
  Recuperación: 1.00
  F1-Score: 1.00
  Documentos encontrados: [318, 4862, 0, 3773, 7727]

Consulta: asian cocoa
  Precisión: 1.00
  Recuperación: 1.00
  F1-Score: 1.00
  Documentos encontrados: [4862, 318, 7727, 319, 366]

Resultados de evaluación guardados en 'evaluacion_sistema.csv'.
